In [1]:
!pip install tqdm

# NO EJECUTEIS EL SIGUIENTE CODIGO, RECOJE LOS DATOS DE LA API Y TARDA MUCHO

In [33]:
import requests
from datetime import datetime, timedelta
import time
import pandas as pd
from tqdm import tqdm

# Define la clave de la API
key = 'zv39NpJBlr56kJJ23JraD9qnQYWPU5tT9nrNDtnc'

# Función para obtener los datos de una variable para una fecha específica
def obtener_datos_variable(codigo_variable, fecha):
    # Construye la URL para obtener los datos de la variable para la fecha especificada
    url = f'https://api.meteo.cat/xema/v1/variables/mesurades/{codigo_variable}/{fecha}?codiEstacio=X8'
    
    # Realiza la solicitud para obtener los datos
    response = requests.get(url, headers={"Content-Type": "application/json", "X-Api-Key": key})
    
    # Verifica si la solicitud fue exitosa
    if response.status_code == 200:
        # Convierte la respuesta JSON en un diccionario de Python
        data = response.json()
        
        # Extrae las lecturas de la variable
        readings = data.get('lectures', [])
        
        # Filtra las lecturas para quedarse solo con las que son en punto
        readings_in_point = [reading for reading in readings if reading['data'].endswith(':00Z')]
        
        # Retorna las lecturas filtradas
        return readings_in_point
    else:
        print(f"Error al obtener los datos para la variable {codigo_variable} en la fecha {fecha}. Código de estado:", response.status_code)
        return None

# Definir la fecha inicial y final
fecha_inicial = datetime(2020, 1, 1)
fecha_final = datetime(2024, 6, 1)

# Calcular el número total de días para el progreso
total_dias = (fecha_final - fecha_inicial).days + 1

# Listas para almacenar los datos
datos_temperatura = []
datos_precipitacion = []
datos_luz = []

# Iterar sobre los días con tqdm para mostrar la barra de progreso
fecha_actual = fecha_inicial
for _ in tqdm(range(total_dias), desc="Extrayendo datos"):
    # Convertir la fecha actual a string en el formato esperado por la API
    fecha_str = fecha_actual.strftime('%Y/%m/%d')
    
    # Obtener los datos de temperatura para la fecha actual
    temperatura = obtener_datos_variable(32, fecha_str)
    if temperatura:
        datos_temperatura.extend(temperatura)
    
    # Obtener los datos de precipitación para la fecha actual
    precipitacion = obtener_datos_variable(35, fecha_str)
    if precipitacion:
        datos_precipitacion.extend(precipitacion)
    
    # Obtener los datos de luz para la fecha actual
    luz = obtener_datos_variable(36, fecha_str)
    if luz:
        datos_luz.extend(luz)
    
    # Sumar un día a la fecha actual
    fecha_actual += timedelta(days=1)
    
    
# Crear DataFrames a partir de los datos recopilados
df_temperatura = pd.DataFrame(datos_temperatura)
df_precipitacion = pd.DataFrame(datos_precipitacion)
df_luz = pd.DataFrame(datos_luz)

# Guardar los DataFrames en archivos CSV
df_temperatura.to_csv('datos_temperatura.csv', index=False)
df_precipitacion.to_csv('datos_precipitacion.csv', index=False)
df_luz.to_csv('datos_luz.csv', index=False)


Extrayendo datos: 100%|██████████| 1614/1614 [36:43<00:00,  1.37s/it] 


In [3]:
import pandas as pd
from tqdm import tqdm

# Lista de archivos CSV
archivos_csv = ['datos_temperatura.csv', 'datos_precipitacion.csv', 'datos_luz.csv']
nombres_columnas = ['temperatura', 'precipitacion', 'luz']

# Diccionario para almacenar los DataFrames
dataframes = {}

# Cargar los archivos CSV en DataFrames con tqdm para mostrar el progreso
for archivo, columna in tqdm(zip(archivos_csv, nombres_columnas), total=len(archivos_csv), desc="Cargando archivos CSV"):
    df = pd.read_csv(archivo)
    print(f"Columnas en {archivo}: {df.columns.tolist()}")
    if 'valor' in df.columns:
        df.rename(columns={'valor': columna}, inplace=True)
    dataframes[columna] = df

# Verificar si todas las columnas necesarias están presentes
for columna in nombres_columnas:
    if 'data' not in dataframes[columna].columns:
        print(f"El DataFrame para {columna} no tiene la columna 'data'.")
        continue

# Filtrar los DataFrames que tienen la columna necesaria
dataframes_validos = {k: v for k, v in dataframes.items() if 'data' in v.columns}

# Combinar los DataFrames usando outer join para mantener todos los datos
df_combined = None
for columna, df in dataframes_validos.items():
    if df_combined is None:
        df_combined = df
    else:
        df_combined = pd.merge(df_combined, df, on='data', how='outer')

# Guardar el DataFrame combinado en un archivo CSV
if df_combined is not None:
    df_combined.to_csv('datos_combinados.csv', index=False)
    # Mostrar una vista previa del DataFrame combinado
    df_combined.head()
else:
    print("No hay DataFrames válidos para combinar.")





Cargando archivos CSV: 100%|██████████| 3/3 [00:00<00:00, 44.78it/s]

Columnas en datos_temperatura.csv: ['data', 'valor', 'estat', 'baseHoraria']
Columnas en datos_precipitacion.csv: ['data', 'valor', 'estat', 'baseHoraria']
Columnas en datos_luz.csv: ['data', 'valor', 'estat', 'baseHoraria']


In [4]:
df_combined

,data,temperatura,estat_x,baseHoraria_x,precipitacion,estat_y,baseHoraria_y,luz,estat,baseHoraria
0,2020-01-01T00:00Z,6.5,V,SH,0.0,V,SH,0.0,V,SH
1,2020-01-01T01:00Z,6.0,V,SH,0.0,V,SH,0.0,V,SH
2,2020-01-01T02:00Z,6.0,V,SH,0.0,V,SH,0.0,V,SH
3,2020-01-01T03:00Z,5.1,V,SH,0.0,V,SH,0.0,V,SH
4,2020-01-01T04:00Z,4.6,V,SH,0.0,V,SH,0.0,V,SH
...,...,...,...,...,...,...,...,...,...,...
38731,2024-06-01T23:00Z,15.2,V,SH,0.0,V,SH,0.0,V,SH
38732,2023-12-22T09:00Z,NaN,NaN,NaN,0.0,V,SH,NaN,NaN,NaN
38733,2023-12-22T10:00Z,NaN,NaN,NaN,0.0,V,SH,NaN,NaN,NaN
38734,2023-12-22T11:00Z,NaN,NaN,NaN,0.0,V,SH,NaN,NaN,NaN


In [5]:
# Convertir la columna 'data' a datetime
df_combined['data'] = pd.to_datetime(df_combined['data'], utc=True)

# Ordenar por la columna 'data'
df_combined = df_combined.sort_values(by='data').reset_index(drop=True)

In [6]:
df_combined

,data,temperatura,estat_x,baseHoraria_x,precipitacion,estat_y,baseHoraria_y,luz,estat,baseHoraria
0,2020-01-01 00:00:00+00:00,6.5,V,SH,0.0,V,SH,0.0,V,SH
1,2020-01-01 01:00:00+00:00,6.0,V,SH,0.0,V,SH,0.0,V,SH
2,2020-01-01 02:00:00+00:00,6.0,V,SH,0.0,V,SH,0.0,V,SH
3,2020-01-01 03:00:00+00:00,5.1,V,SH,0.0,V,SH,0.0,V,SH
4,2020-01-01 04:00:00+00:00,4.6,V,SH,0.0,V,SH,0.0,V,SH
...,...,...,...,...,...,...,...,...,...,...
38731,2024-06-01 19:00:00+00:00,18.3,V,SH,0.3,V,SH,0.0,V,SH
38732,2024-06-01 20:00:00+00:00,16.8,V,SH,0.1,V,SH,0.0,V,SH
38733,2024-06-01 21:00:00+00:00,16.6,V,SH,1.1,V,SH,0.0,V,SH
38734,2024-06-01 22:00:00+00:00,15.4,V,SH,0.0,V,SH,0.0,V,SH


In [14]:
# Seleccionar solo las columnas de interés y hacer una copia
df_filtered = df_combined[['data', 'temperatura', 'precipitacion', 'luz']].copy()

# Eliminar filas con cualquier NaN en las columnas seleccionadas
df_clean = df_filtered.dropna(how='any').reset_index(drop=True)

# Verificar si hay cualquier NaN en el DataFrame limpio
any_nans = df_clean.isna().any().any()
print("¿Hay algún NaN en el DataFrame limpio?", any_nans)

# Contar los NaNs en cada columna
nans_per_column = df_clean.isna().sum()
print("\nRecuento de NaNs por columna:")
print(nans_per_column)



¿Hay algún NaN en el DataFrame limpio? False

Recuento de NaNs por columna:
data             0
temperatura      0
precipitacion    0
luz              0
dtype: int64


In [13]:
df_clean

,data,temperatura,precipitacion,luz
0,2020-01-01 00:00:00+00:00,6.5,0.0,0.0
1,2020-01-01 01:00:00+00:00,6.0,0.0,0.0
2,2020-01-01 02:00:00+00:00,6.0,0.0,0.0
3,2020-01-01 03:00:00+00:00,5.1,0.0,0.0
4,2020-01-01 04:00:00+00:00,4.6,0.0,0.0
...,...,...,...,...
38727,2024-06-01 19:00:00+00:00,18.3,0.3,0.0
38728,2024-06-01 20:00:00+00:00,16.8,0.1,0.0
38729,2024-06-01 21:00:00+00:00,16.6,1.1,0.0
38730,2024-06-01 22:00:00+00:00,15.4,0.0,0.0


In [16]:
# Obtener los valores máximos de cada columna
max_values = df_clean.max()
max_values

data             2024-06-01 23:00:00+00:00
temperatura                           37.4
precipitacion                         15.6
luz                                 1073.0
dtype: object

In [20]:
# Crear las columnas booleanas
df_clean['calor'] = df_clean['temperatura'] > 30
df_clean['lluvia'] = df_clean['precipitacion'] > 1
df_clean['dia'] = df_clean['luz'] > 10

# Convertir booleanos a 1 y 0
df_clean['calor'] = df_clean['calor'].astype(int)
df_clean['lluvia'] = df_clean['lluvia'].astype(int)
df_clean['dia'] = df_clean['dia'].astype(int)

In [21]:
df_clean

,data,temperatura,precipitacion,luz,calor,lluvia,dia
0,2020-01-01 00:00:00+00:00,6.5,0.0,0.0,0,0,0
1,2020-01-01 01:00:00+00:00,6.0,0.0,0.0,0,0,0
2,2020-01-01 02:00:00+00:00,6.0,0.0,0.0,0,0,0
3,2020-01-01 03:00:00+00:00,5.1,0.0,0.0,0,0,0
4,2020-01-01 04:00:00+00:00,4.6,0.0,0.0,0,0,0
...,...,...,...,...,...,...,...
38727,2024-06-01 19:00:00+00:00,18.3,0.3,0.0,0,0,0
38728,2024-06-01 20:00:00+00:00,16.8,0.1,0.0,0,0,0
38729,2024-06-01 21:00:00+00:00,16.6,1.1,0.0,0,1,0
38730,2024-06-01 22:00:00+00:00,15.4,0.0,0.0,0,0,0


In [22]:
# Encontrar los valores de 1 y 0 en la columna 'calor'
calor_1 = df_clean[df_clean['calor'] == 1].shape[0]
calor_0 = df_clean[df_clean['calor'] == 0].shape[0]

# Encontrar los valores de 1 y 0 en la columna 'lluvia'
lluvia_1 = df_clean[df_clean['lluvia'] == 1].shape[0]
lluvia_0 = df_clean[df_clean['lluvia'] == 0].shape[0]

# Encontrar los valores de 1 y 0 en la columna 'dia'
dia_1 = df_clean[df_clean['dia'] == 1].shape[0]
dia_0 = df_clean[df_clean['dia'] == 0].shape[0]

print("Valores en la columna 'calor':")
print("1:", calor_1)
print("0:", calor_0)

print("\nValores en la columna 'lluvia':")
print("1:", lluvia_1)
print("0:", lluvia_0)

print("\nValores en la columna 'dia':")
print("1:", dia_1)
print("0:", dia_0)

Valores en la columna 'calor':
1: 903
0: 37829

Valores en la columna 'lluvia':
1: 251
0: 38481

Valores en la columna 'dia':
1: 18601
0: 20131


In [24]:
df_meteo=df_clean[['data','calor','lluvia','dia']]
df_meteo

,data,calor,lluvia,dia
0,2020-01-01 00:00:00+00:00,0,0,0
1,2020-01-01 01:00:00+00:00,0,0,0
2,2020-01-01 02:00:00+00:00,0,0,0
3,2020-01-01 03:00:00+00:00,0,0,0
4,2020-01-01 04:00:00+00:00,0,0,0
...,...,...,...,...
38727,2024-06-01 19:00:00+00:00,0,0,0
38728,2024-06-01 20:00:00+00:00,0,0,0
38729,2024-06-01 21:00:00+00:00,0,1,0
38730,2024-06-01 22:00:00+00:00,0,0,0


In [25]:
df_meteo.to_csv('valores_booleanos_meteo.csv', index=False)